In [1]:
import pandas as pd
import numpy as np
from google.colab import files

In [2]:
uploaded = files.upload()     #AllNodeAttributes.csv 

Saving AllNodeAttributes.csv to AllNodeAttributes.csv


In [3]:
!pip install python-igraph
from igraph import *

     |████████████████████████████████| 3.2 MB 5.1 MB/s 
  Created wheel for python-igraph: filename=python_igraph-0.9.8-py3-none-any.whl size=9070 sha256=4cca500f21cbbb12084552c0761c1195f3aff499b9dd403a8d62e624c03088c5
  Stored in directory: /root/.cache/pip/wheels/15/86/ef/b8bcdfbcb1c489771ad256c7cd1eb4971cdb7f3f670938b798
Successfully built python-igraph


In [5]:
NodeAttributes= pd.read_csv("AllNodeAttributes.csv") 
#NodeAttributes['State'].replace('', np.nan, inplace=True)
#NodeAttributes.dropna(subset=['State'], inplace=True)    #remove all entries where State field is blank #906 users out of 943 left
print("Total nodes are:", NodeAttributes['UserID'].count())
NodeAttributes.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)   #There was an unamed column in the file, needed to delete that column
NodeAttributes.drop(["a"], axis=1, inplace=True)
NodeAttributes = NodeAttributes.drop_duplicates()
print("After dropping duplicates Total nodes are:" , NodeAttributes['UserID'].nunique())
NodeAttributes.head()
NodeAttributes.loc[NodeAttributes['UserID'] == 196]    

NodeAttributes['GenderType'] = np.where(NodeAttributes['Gender'] == 1, 'M','F')

NodeAttributes['AgeGroup'] = NodeAttributes['AgeGroup'].fillna('Teenager')
NodeAttributes['State'] = NodeAttributes['State'].fillna('TX')
print(NodeAttributes['Gender'].value_counts(dropna=False))
NodeAttributes.head()

Total nodes are: 943
After dropping duplicates Total nodes are: 943
1    670
0    273
Name: Gender, dtype: int64


,UserID,Age,Gender,Occupation,Zipcode,NormalizedAge,AgeGroup,Male,Female,State,Teenager,Youth,MiddleAge,Senior,SuperSenior,administrator,artist,doctor,educator,engineer,entertainment,executive,healthcare,homemaker,lawyer,librarian,marketing,none,other,programmer,retired,salesman,scientist,student,technician,writer,AZ,CA,FL,OH,...,Western,ActionRating,AdventureRating,AnimationRating,ChildrenRating,ComedyRating,CrimeRating,DocumentaryRating,DramaRating,FantasyRating,FilmNoirRating,HorrorRating,MusicalRating,MysteryRating,RomanceRating,SciFiRating,ThrillerRating,WarRating,WesternRating,NumMoviesWatched,unknownPreference,ActionPreference,AdventurePreference,AnimationPreference,ChildrenPreference,ComedyPreference,CrimePreference,DocumentaryPreference,DramaPreference,FantasyPreference,FilmNoirPreference,HorrorPreference,MusicalPreference,MysteryPreference,RomancePreference,SciFiPreference,ThrillerPreference,WarPreference,WesternPreference,GenderType
0,1,24,1,technician,85711,0.328767,Youth,1,0,AZ,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0.010221,0.666667,0.585714,0.666667,0.440000,0.694505,0.688000,0.96,0.785047,0.7,1.0,0.692308,0.584615,0.720000,0.786364,0.800000,0.723077,0.736000,0.733333,587.0,1,1,0,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,M
1,2,53,0,other,94043,0.726027,Senior,0,1,CA,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0.000000,0.760000,0.866667,0.800000,0.600000,0.762500,0.755556,0.00,0.765714,0.6,0.9,0.600000,0.600000,0.700000,0.825000,0.750000,0.716667,0.733333,0.000000,123.0,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,F
2,3,23,1,writer,32067,0.315068,Youth,1,0,FL,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,...,0.000000,0.557143,0.700000,0.000000,0.000000,0.516667,0.600000,1.00,0.581818,0.0,0.5,0.480000,0.400000,0.636364,0.680000,0.550000,0.504762,0.560000,0.000000,122.0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,M
3,4,24,1,technician,43537,0.328767,Youth,1,0,OH,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,...,0.000000,0.775000,0.700000,0.000000,0.000000,1.000000,0.950000,1.00,0.900000,0.0,0.0,0.800000,1.000000,0.800000,0.866667,0.766667,0.781818,0.900000,0.000000,56.0,0,1,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,1,0,M
4,5,33,0,other,15213,0.452055,MiddleAge,0,1,PA,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0.005208,0.628571,0.648485,0.757143,0.489655,0.600000,0.777778,0.00,0.533333,0.5,1.0,0.507143,0.666667,0.600000,0.463158,0.703030,0.589474,0.642857,0.500000,384.0,1,1,1,1,0,1,1,0,0,0,1,0,1,1,0,1,0,1,0,F


In [6]:
#Creating groups of column names
UnNormalizedFeatures=['Age', 'Occupation',	'Zipcode',	'AgeGroup',	'State','NumMoviesWatched']
NumericFeatures = ['NormalizedAge']
AllStates = pd.unique(NodeAttributes['State']).tolist()
AllStates = [x for x in AllStates if str(x) != 'nan']
AllOccupations = pd.unique(NodeAttributes['Occupation'])
AgeLabels = ['Teenager','Youth','MiddleAge','Senior','SuperSenior']
AllGenres= ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir',
       'Horror', 'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller', 'War', 'Western']
AllGenresRatings = []
for genre in AllGenres:
   AllGenresRatings.append(genre+'Rating')
AllGenresPreference = []
for genre in AllGenres:
   AllGenresPreference.append(genre+'Preference')

In [11]:
Tier1Columns =  ['Male','Female', 'NormalizedAge'] #['Gender','NormalizedAge'] 
#Tier1Columns.extend(AgeLabels)
Tier1Columns.extend(AllOccupations)
Tier1Columns.extend(AllStates)
Tier1Attributes = NodeAttributes[Tier1Columns]
#Tier1Attributes

Tier2Columns = []
Tier2Columns.extend(AllGenres)
Tier2Columns.extend(AllGenresPreference)
#Tier2Columns.extend(AllGenresRatings)      #with only genreratings no community is split
print('Tier1 Features: ',Tier1Columns)  
print('Tier2 Features: ',Tier2Columns) 

Tier1 Features:  ['Male', 'Female', 'NormalizedAge', 'technician', 'other', 'writer', 'executive', 'administrator', 'student', 'lawyer', 'educator', 'scientist', 'entertainment', 'programmer', 'librarian', 'homemaker', 'artist', 'engineer', 'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor', 'AZ', 'CA', 'FL', 'OH', 'PA', 'WA', 'VT', 'MA', 'GA', 'CT', 'SC', 'MN', 'OR', 'NY', 'TN', 'KY', 'MI', 'MD', 'TX', 'NC', 'WI', 'CO', 'IN', 'IA', 'NJ', 'UT', 'IL', 'VA', 'OK', 'NH', 'DC', 'NV', 'NE', 'LA', 'MO', 'KS', 'AK', 'AL', 'ID', 'ND', 'MS', 'AR', 'MT', 'NM', 'RI', 'HI', 'WV', 'ME', 'DE', 'WY', 'SD']
Tier2 Features:  ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir', 'Horror', 'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller', 'War', 'Western', 'ActionPreference', 'AdventurePreference', 'AnimationPreference', 'ChildrenPreference', 'ComedyPreference', 'CrimePreference', 'DocumentaryPreference', 'DramaPreference', 

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import stats
similarityMatrix = cosine_similarity(Tier1Attributes)     #cosine_similarity(Tier1Attributes, Tier1Attributes)
np.fill_diagonal(similarityMatrix, 0)    #to remove self loops
print('mean median mode: ',similarityMatrix.mean(), np.median(similarityMatrix),stats.mode(similarityMatrix,axis=None))
print(similarityMatrix.shape)
#similarityMatrix
thresh = 0.5 #np.median(similarityMatrix) #0.5                  
#values_morethanequalto_thresh = (cosinematrix >= thresh)
#values_morethanequalto_thresh.sum()
adjacencymatrix = similarityMatrix.copy()
adjacencymatrix[adjacencymatrix >= thresh] = 1
adjacencymatrix[adjacencymatrix < thresh] = 0
adjacencymatrix
values_equalto_one = (adjacencymatrix == 1)
values_equalto_one.sum()

mean median mode:  0.2942004718747698 0.3605351773774652 ModeResult(mode=array([0.36294844]), count=array([1656]))
(943, 943)


74574

In [9]:
#print( NodeAttributes.UserID )
print("Total nodes are:", NodeAttributes['UserID'].count())
print(adjacencymatrix.shape)
graph = Graph.Adjacency(adjacencymatrix, mode="undirected")  
graph.vs['name'] = NodeAttributes.UserID 
graph.vs['Age'] = NodeAttributes.Age 
graph.vs['Gender'] = NodeAttributes.Gender
Occupation_codes = NodeAttributes.Occupation.astype('category').cat.codes  #converting to numeric category
graph.vs['Occupation'] = Occupation_codes
Age_codes = NodeAttributes.AgeGroup.astype('category').cat.codes #converting to numeric category
graph.vs['AgeGroup'] = Age_codes
State_codes = NodeAttributes.State.astype('category').cat.codes  #converting to numeric category
graph.vs['State'] = State_codes
Gender_codes = NodeAttributes.GenderType.astype('category').cat.codes  #converting to numeric category
graph.vs['GenderType'] = Gender_codes

print(graph.vcount(),graph.ecount())
graph=graph.simplify()
print(graph.vcount(),graph.ecount())
#summary(graph)
len(graph.vs['AgeGroup'])

graph.vs.select(name=18)['Gender']

Total nodes are: 943
(943, 943)
943 37287
943 37287


[0]

In [10]:
print('Full Graph: vcount ecount: ',graph.vcount(),graph.ecount())

communities = Graph.community_multilevel(graph)
print('Number of communities: ', len(communities))
print('modularity: ', graph.modularity(communities))
membership = communities.membership
#print('Number of communities: ',len(communities))
for n in range(0,len(communities)):
  print ('Community nº', n, 'size:', len(communities[n]))
  communityMembers_id = communities[n]
  communityMembers = [graph.vs[x]["name"] for x in communities[n]]
 
  communitygraph = graph.subgraph(communityMembers_id)
  communitygraph.vs['name'] = communityMembers

  #print(len(communitygraph.vs['Gender']))
  #print(communitygraph.vs['Gender'])
  print('Community Graph: vcount ecount: ',communitygraph.vcount(),communitygraph.ecount())

  CommunityGraphNodeAttributes = NodeAttributes[NodeAttributes['UserID'].isin(communityMembers)]
  communitygraph.vs['Age'] =  list(CommunityGraphNodeAttributes.Age )
  communitygraph.vs['Gender'] = list(CommunityGraphNodeAttributes.Gender)
  #print(communitygraph.vs['Gender'] )
  Occupation_codes = CommunityGraphNodeAttributes.Occupation.astype('category').cat.codes  #converting to numeric category
  communitygraph.vs['Occupation'] = list(Occupation_codes)
  Age_codes = CommunityGraphNodeAttributes.AgeGroup.astype('category').cat.codes #converting to numeric category
  communitygraph.vs['AgeGroup'] = list(Age_codes)
  State_codes = CommunityGraphNodeAttributes.State.astype('category').cat.codes  #converting to numeric category
  communitygraph.vs['State'] =list(State_codes)

  ag = GraphBase.assortativity_nominal(communitygraph, communitygraph.vs["Gender"], directed = 'FALSE')
  agt = GraphBase.assortativity_nominal(communitygraph, communitygraph.vs["GenderType"], directed = 'FALSE')
  ao = GraphBase.assortativity_nominal(communitygraph, communitygraph.vs["Occupation"], directed = 'FALSE')
  aa = GraphBase.assortativity_nominal(communitygraph, communitygraph.vs["AgeGroup"], directed = 'FALSE')
  ast = GraphBase.assortativity_nominal(communitygraph, communitygraph.vs["State"], directed = 'FALSE')
  print('CommunityGraph-Tier1: assortativity: Gender: ', ag, ' GenderType: ', agt,' Occupation: ', ao, ' Age: ', aa, ' State:', ast)


  NodesInCommunity = NodeAttributes.loc[NodeAttributes['UserID'].isin(communityMembers)][Tier2Columns]

  cosineSimilarity = cosine_similarity(NodesInCommunity)
  thresh = 0.5
  adjacencyMatrix = cosineSimilarity.copy()
  adjacencyMatrix[adjacencyMatrix >= thresh] = 1
  adjacencyMatrix[adjacencyMatrix < thresh] = 0
  adjacencyMatrix_df = pd.DataFrame(adjacencyMatrix, columns=NodesInCommunity.index, index=NodesInCommunity.index)
  subgraph = Graph.Adjacency(adjacencyMatrix, mode="undirected")  
  subgraph.vs['name'] = communityMembers    #NodeAttributesWithCommunity.UserID 
  #print('vertices: ',  subgraph.vs['name'] )
  #print(subgraph.vcount(),subgraph.ecount())
  subgraph=subgraph.simplify()
  print('CommunityGraph-Tier2: vcount ecount: ',subgraph.vcount(),subgraph.ecount())
  subcommunities = Graph.community_multilevel(subgraph)
  submembership = subcommunities.membership
  print('Number of subcommunities: ',len(subcommunities))
  print('modularity: ', subgraph.modularity(subcommunities))

  for n in range(0,len(subcommunities)):
    print ('Community nº', n, 'size:', len(subcommunities[n]))
    nodes_in_subcommunity_ids = subcommunities[n]
    nodes_in_subcommunity = [subgraph.vs[x]["name"] for x in subcommunities[n]]
    
    #print('communityMembers: ', communityMembers)
    #print('nodes_in_subcommunity: ', nodes_in_subcommunity)

    SubGraphNodeAttributes = NodeAttributes[NodeAttributes['UserID'].isin(nodes_in_subcommunity)]   #etract attributes of nodes with attribte 'Name' matchinh UserID
    #print('Shape: ' ,SubGraphNodeAttributes.shape)
    #print('SubGraphNodeAttributes: ', SubGraphNodeAttributes['UserID'].tolist())
    #print(SubGraphNodeAttributes.head())

    subsubgraph = subgraph.subgraph(nodes_in_subcommunity_ids)  #make sub graph with vertex ids
    print('subsubgraph vcount ecount: ',subsubgraph.vcount(),subsubgraph.ecount())
    
    subsubgraph.vs['name'] =  list(SubGraphNodeAttributes.UserID)
    subsubgraph.vs['Age'] =  list(SubGraphNodeAttributes.Age )
    subsubgraph.vs['Gender'] = list(SubGraphNodeAttributes.Gender)
    #print(subsubgraph.vs['Gender'] )
    Occupation_codes = SubGraphNodeAttributes.Occupation.astype('category').cat.codes  #converting to numeric category
    subsubgraph.vs['Occupation'] = list(Occupation_codes)
    Age_codes = SubGraphNodeAttributes.AgeGroup.astype('category').cat.codes #converting to numeric category
    subsubgraph.vs['AgeGroup'] = list(Age_codes)
    State_codes = SubGraphNodeAttributes.State.astype('category').cat.codes  #converting to numeric category
    subsubgraph.vs['State'] =list(State_codes)

    ag = GraphBase.assortativity_nominal(subsubgraph, subsubgraph.vs["Gender"], directed = 'FALSE')
    ao = GraphBase.assortativity_nominal(subsubgraph, subsubgraph.vs["Occupation"], directed = 'FALSE')
    aa = GraphBase.assortativity_nominal(subsubgraph, subsubgraph.vs["AgeGroup"], directed = 'FALSE')
    ast = GraphBase.assortativity_nominal(subsubgraph, subsubgraph.vs["State"], directed = 'FALSE')
    print('SubCommunity:  assortativity: Gender: ', ag, ' Occupation: ', ao, ' Age: ', aa, ' State:', ast)

    for attribute in Tier2Columns:
      attribute_codes = SubGraphNodeAttributes[attribute].astype('category').cat.codes  #converting to numeric category
      subsubgraph.vs[attribute+'Preference'] =  list(attribute_codes)
      print('assortativity:',attribute,' = ', GraphBase.assortativity_nominal(subsubgraph, subsubgraph.vs[attribute+'Preference'], directed = 'FALSE'))
  








Full Graph: vcount ecount:  943 37287
Number of communities:  7
modularity:  0.545385716419724
Community nº 0 size: 145
Community Graph: vcount ecount:  145 3034
CommunityGraph-Tier1: assortativity: Gender:  nan  GenderType:  nan  Occupation:  0.7234240313063064  Age:  0.033861527641435386  State: 0.09134317620404486
CommunityGraph-Tier2: vcount ecount:  145 10301
Number of subcommunities:  2
modularity:  0.0031313742158523006
Community nº 0 size: 74
subsubgraph vcount ecount:  74 2700
SubCommunity:  assortativity: Gender:  nan  Occupation:  -0.013721443210263128  Age:  -0.013610603366372263  State: -0.013705290740425799
assortativity: Action  =  -0.013688815115557853
assortativity: Adventure  =  -0.013699255851832116
assortativity: Animation  =  -0.013914594096341323
assortativity: Children  =  -0.013658987126762119
assortativity: Comedy  =  -0.013699116411689704
assortativity: Crime  =  -0.013693965737452985
assortativity: Documentary  =  -0.013806640471267688
assortativity: Drama  =